In [1]:
import pandas as pd
import sqlite3
import random
import os

conn = sqlite3.connect('wikireddit_final_notopics.db')

In [2]:
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")

for table in cursor.fetchall():
    cursor.execute(f"PRAGMA table_info({table[0]})")
    print(table)
    display(pd.DataFrame(cursor.fetchall()))

('posts',)


,0,1,2,3,4,5
0,0,subreddit_id,TEXT,0,None,0
1,1,crosspost_parent_id,TEXT,0,None,0
2,2,id,TEXT,0,None,0
3,3,created_at,TIMESTAMP,0,None,0
4,4,updated_at,TIMESTAMP,0,None,0
5,5,language_code,TEXT,0,None,0
6,6,score,INTEGER,0,None,0
7,7,upvote_ratio,REAL,0,None,0
8,8,gildings,INTEGER,0,None,0
9,9,num_comments,INTEGER,0,None,0


('comments',)


,0,1,2,3,4,5
0,0,subreddit_id,TEXT,0,None,0
1,1,post_id,TEXT,0,None,0
2,2,parent_id,TEXT,0,None,0
3,3,id,TEXT,0,None,0
4,4,created_at,TIMESTAMP,0,None,0
5,5,last_modified_at,TIMESTAMP,0,None,0
6,6,score,INTEGER,0,None,0
7,7,upvote_ratio,REAL,0,None,0
8,8,gilded,INTEGER,0,None,0


('wiki_ids',)


...

,0,1,2,3,4,5
0,0,final_url,TEXT,0,None,0
1,1,lang,TEXT,0,None,0
2,2,mobile,INTEGER,0,None,0
3,3,raw_title,TEXT,0,None,0


('postlinks',)


,0,1,2,3,4,5
0,0,id,TEXT,0,None,0
1,1,end_processed_valid,INTEGER,0,None,0
2,2,end_processed_url,TEXT,0,None,0
3,3,final_valid,INTEGER,0,None,0
4,4,final_status,INTEGER,0,None,0
5,5,final_url,TEXT,0,None,0
6,6,in_title,INTEGER,0,None,0
7,7,redirected,INTEGER,0,None,0


('commentlinks',)


,0,1,2,3,4,5
0,0,id,TEXT,0,None,0
1,1,end_processed_valid,INTEGER,0,None,0
2,2,end_processed_url,TEXT,0,None,0
3,3,final_valid,INTEGER,0,None,0
4,4,final_status,INTEGER,0,None,0
5,5,final_url,TEXT,0,None,0
6,6,redirected,INTEGER,0,None,0


Rename cols

In [ ]:
renamedict = {}

Get sample dataset

In [3]:
conn_sample = sqlite3.connect('wikireddit_sample.db')

In [4]:
posts = pd.read_sql("SELECT * FROM posts", con=conn)
comments = pd.read_sql("SELECT * FROM comments", con=conn)

postixs = sorted([random.randint(0, len(posts)) for _ in range(200)])
commentixs = sorted([random.randint(0, len(comments)) for _ in range(2000)])

posts_sample = posts.iloc[postixs]
comments_sample = comments.iloc[commentixs]

posts_sample_ids = posts_sample['id']
comments_sample_ids = comments_sample['id']

In [5]:
postlinks = pd.read_sql("SELECT * FROM postlinks", con=conn)
commentlinks = pd.read_sql("SELECT * FROM commentlinks", con=conn)

In [6]:
postlinks_sample = postlinks[postlinks['id'].isin(posts_sample_ids)]
commentlinks_sample = commentlinks[commentlinks['id'].isin(comments_sample_ids)]
links_sample = set(postlinks_sample['final_url']) | set(commentlinks_sample['final_url'])

In [7]:
linkarticles = pd.read_sql("SELECT * FROM linkarticles", con=conn)
linkarticles_sample = linkarticles[linkarticles['final_url'].isin(links_sample)]

In [8]:
resolved_redirects = pd.read_sql("SELECT * FROM resolved_redirects", con=conn)
collected_redirects = pd.read_sql("SELECT * FROM collected_redirects", con=conn)

resolved_redirects_sample = linkarticles_sample.merge(resolved_redirects, on=['lang', 'raw_title'], how='left')[resolved_redirects.columns]
resolved_redirects_sample = resolved_redirects_sample.rename({'redirected_title': 'canonical_title'}, axis=1)
collected_redirects_sample = resolved_redirects_sample.merge(collected_redirects, on=['lang', 'canonical_title'], how='left')

In [9]:
wiki_ids = pd.read_sql("SELECT * FROM wiki_ids", con=conn)
wiki_ids_sample = resolved_redirects_sample.merge(wiki_ids, left_on=['lang', 'canonical_title'],
                                right_on=['lang', 'title'], how='left')[wiki_ids.columns]

In [10]:
# pageviews = pd.read_sql("SELECT * FROM pageviews", con=conn)
# pageviews_sample = resolved_redirects_sample.merge(pageviews, left_on=['lang', 'canonical_title'],
#                                 right_on=['lang', 'title'], how='left')[pageviews.columns]


In [11]:
# revisions = pd.read_sql("SELECT * FROM revisions", con=conn)
# revisions_sample = resolved_redirects_sample.merge(revisions, left_on=['lang', 'canonical_title'],
#                                 right_on=['lang', 'title'], how='left')[revisions.columns]


In [12]:
# topics = pd.read_sql("SELECT * FROM topics", con=conn)
# topics_sample = resolved_redirects_sample.merge(topics, left_on=['lang', 'canonical_title'],
#                                 right_on=['lang', 'title'], how='left')[topics.columns]

In [13]:
posts_sample.to_sql('posts', con=conn_sample, if_exists='replace')
comments_sample.to_sql('comments_sample', con=conn_sample, if_exists='replace')

postlinks_sample.to_sql('postlinks', con=conn_sample, if_exists='replace')
commentlinks_sample.to_sql('commentlinks', con=conn_sample, if_exists='replace')
linkarticles_sample.to_sql('linkarticles', con=conn_sample, if_exists='replace')

resolved_redirects_sample.to_sql('resolved_redirects', con=conn_sample, if_exists='replace')
collected_redirects_sample.to_sql('collected_redirects_sample', con=conn_sample, if_exists='replace')
wiki_ids_sample.to_sql('wiki_ids', con=conn_sample, if_exists='replace')

# pageviews_sample.to_sql('pageviews', con=conn_sample, if_exists='replace')
# revisions_sample.to_sql('revisions', con=conn_sample, if_exists='replace')
# topics_sample.to_sql('topics', con=conn_sample, if_exists='replace')

2817

In [14]:
mb_size = os.path.getsize('wikireddit_sample.db') / 1024 / 1024
print(f"Database size: {mb_size:.2f} MB")

Database size: 5.30 MB
